In [26]:
import requests
from dataflows import Flow, printer, checkpoint, update_resource, dump_to_path


def bh_search(query, collection):
    print(f'running query: {query} ({collection})')
    res = requests.get('https://api.dbs.bh.org.il/v1/search', params={
        'q': query,
        'collection': collection,
        'from': 0, 'size': 9999
    })
    for row in res.json()['hits']['hits']:
        yield row['_source']

        
Flow(
    bh_search('אתיופיה OR ethiopia', 'familyNames'), update_resource('res_1', name='familyNames', path='familyNames.csv'),
    bh_search('אתיופיה OR ethiopia', 'places'), update_resource('res_2', name='places', path='places.csv'),
    bh_search('אתיופיה OR ethiopia', 'photoUnits'), update_resource('res_3', name='photoUnits', path='photoUnits.csv'),
    bh_search('אתיופיה OR ethiopia', 'personalities'), update_resource('res_4', name='personalities', path='personalities.csv'),
    dump_to_path('data/mojp-dbs-items'),
    checkpoint('mojp-dbs-items'),
    printer(tablefmt='html', num_rows=1)
).process()[1]

saving checkpoint to: .checkpoints/mojp-dbs-items
running query: אתיופיה OR ethiopia (familyNames)
running query: אתיופיה OR ethiopia (places)
running query: אתיופיה OR ethiopia (photoUnits)
running query: אתיופיה OR ethiopia (personalities)


#,LocationInMuseum (any),Pictures (array),PictureUnitsIds (any),UpdateDate (string),OldUnitId (string),UpdateUser (string),UnitDisplayStatus (integer),PrevPicturePaths (any),TS (string),UnitType (integer),UnitTypeDesc (string),EditorRemarks (string),UnitStatus (integer),RightsDesc (string),Bibiliography (object),UnitText1 (object),UnitText2 (object),UnitPlaces (array),DisplayStatusDesc (string),UnitHeaderDMSoundex (object),PrevPictureFileNames (any),RightsCode (integer),UnitId (integer),IsValueUnit (integer),StatusDesc (string),UserLexicon (any),Attachments (array),Slug (object),Header (object),ForPreview (integer),Id (integer)
1,None,[],None,2012-11-30T10:22:00,FM014459.HTM,Haim - Family Names (2012),2,None,000000000002a3e8,6,Family Name,hasavot from Family Names,3,Full,"{'En': None, 'He': None}","{'En': ""HABESHA\r\n\r\nThis name is of Ethiopian origin. Habesha is a female personal name. It is t ...","{'En': None, 'He': None}",[],Museum only,"{'En': 'ZZ E574000 ZZ ', 'He': 'ZZ E574000 ZZ '}",None,1,79472,True,Completed,None,[],"{'En': 'familyname_habesha', 'He': 'שםמשפחה_חבשה'}","{'En': 'HABESHA', 'He': 'חבשה'}",False,342973
2,None,[],None,2012-11-30T10:20:00,FM014100.HTM,Haim - Family Names (2012),2,None,0000000000029391,6,Family Name,hasavot from Family Names,3,Full,"{'En': None, 'He': None}","{'En': 'AREGITU, AREGO, AREGNESH\r\n\r\nThis name is of Ethiopian origin. Aregitu is a male persona ...","{'En': None, 'He': None}",[],Museum only,"{'En': 'ZZ E095300 ZZ ', 'He': 'ZZ E095300 ZZ '}",None,1,79114,True,Completed,None,[],"{'En': 'familyname_aregitu', 'He': 'שםמשפחה_ארגיטו'}","{'En': 'AREGITU', 'He': 'ארגיטו'}",False,338027
...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1226,None,[],None,2015-11-09T09:22:00,FM003081.HTM,Paz,2,None,00000000004515aa,6,Family Name,hasavot from Family Names,3,Full,"{'En': None, 'He': None}","{'En': 'MARCUS, MARKUS\n\nSurnames derive from one of many different origins. \n\nSometimes there ma ...","{'En': None, 'He': None}",[],Museum only,"{'En': 'ZZ E695400 ZZ ', 'He': 'ZZ E695400 ZZ '}",None,1,90082,True,Completed,None,[],"{'En': 'familyname_markus', 'He': 'שםמשפחה_מרקוס-90082'}","{'En': 'MARKUS', 'He': 'מרקוס'}",False,346389


#,LocationInMuseum (any),Pictures (array),PictureUnitsIds (string),related (array),UpdateDate (string),OldUnitId (string),UpdateUser (string),PrevPictureFileNames (string),PrevPicturePaths (string),PlaceTypeCode (integer),TS (string),PlaceTypeDesc (object),UnitType (integer),UnitTypeDesc (string),EditorRemarks (string),DisplayStatusDesc (string),RightsDesc (string),Bibiliography (object),UnitText1 (object),UnitText2 (object),UnitPlaces (array),main_image_url (string),UnitStatus (integer),UnitHeaderDMSoundex (object),UnitDisplayStatus (integer),RightsCode (integer),UnitId (integer),IsValueUnit (integer),StatusDesc (string),PlaceParentTypeCodeDesc (object),PlaceParentId (integer),UserLexicon (any),Attachments (array),geometry (object),Slug (object),Header (object),thumbnail_url (string),ForPreview (integer)
1,None,[{'PictureUrl': 'https://storage.googleapis.com/bhs-flat-pics/BD900F09-78CF-490B-B451-0470FFAC5633.j ...,10981|7953|10971|10515|6200|,"['image_the-kohenat-reciting-prayers-on-the-mountain-near-ambober-ethiopia-1956', 'image_interior-of ...",2016-06-23T11:08:00,HB068228.HTM-EB068300.HTM,simona,01049901.JPG|00746701.JPG|01048900.JPG|01003204.JPG|00571300.JPG|,Photos\00000991.scn\01049901.JPG|Photos\00000521.scn\00746701.JPG|Photos\00000151.scn\01048900.JPG|P ...,6,     ];Y,"{'En': 'Country', 'He': 'ארץ'}",5,Place,hasavot from Places,Museum only,Full,"{'En': 'ENCYCLOPAEDIA HEBRAICA \nLIST, MIRIAM NIR', 'He': ""אנציקלופדיה עברית, מילואים ג' \nמקורות, ...",{'En': 'ETHIOPIA\r\n\r\n(FORMERLY ABYSSINIA)\r\n\r\nA STATE IN NORTH- EAST AFRICA.\r\n\r\nTHE JEWS O ...,"{'En': None, 'He': None}",[],https://storage.googleapis.com/bhs-flat-pics/758A8DAD-FF42-46BB-8710-C94D8ABF9714.jpg,3,"{'En': 'ZZ E037000 ZZ ', 'He': 'ZZ H031700 H037000 H041700 H047000 ZZ '}",2,1,73463,True,Completed,{'En': 'Continent'},66811,None,[],"{'type': 'Point', 'coordinates': [38.6521203, 10.2116702]}","{'En': 'place_ethiopia', 'He': 'מקום_אתיופיה'}","{'En': 'ETHIOPIA', 'He': 'אתיופיה'}",https://storage.googleapis.com/bhs-thumbnails/758A8DAD-FF42-46BB-8710-C94D8ABF9714.jpg,False
2,None,[{'PictureUrl': 'https://storage.googleapis.com/bhs-flat-pics/E8D5A6FB-04F7-44B7-9C3B-0386F8823A7D.j ...,41275|9891|146063|146066|5316|20305|9986|3526|20303|48063|9985|9988|9873|9983|9987|48062|9984|3521|2 ...,"['image_brit-milah-ceremony-of-nahum-daniel-nairobi-kenya-1989', 'image_freidrich-lichtenstein-rabbi ...",2016-10-27T15:32:00,HB000686.HTM-EB001477.HTM,racheld,01839600.JPG|00940800.JPG|Forward Collection VIII-7.jpg|Forward Collection VIII-8.jpg|00482501.JPG|0 ...,Photos\00001037.scn\01839600.JPG|Photos\00000139.scn\00940800.JPG|Photos\805e5c90-4a10-471e-aa21-33c ...,6,00000000006efce0,"{'En': 'Country', 'He': 'ארץ'}",5,Place,hasavot from Places,Museum only,Full,"{'En': 'Encyclopedia Judaica\n\nNairobi Hebrew Congregation, 1991\n\nhttps://nairobisynagogue.org/\n ...","{'En': ""Kenya\n\nA country in east Africa\n\nKenya’s capital and largest city is Nairobi.\n\nThe Nai ...","{'En': None, 'He': None}",[],https://storage.googleapis.com/bhs-flat-pics/F71E3BB9-E61A-4994-B10A-4AF94CBE7C41.jpg,3,"{'En': 'ZZ E560000 ZZ ', 'He': 'ZZ H560000 ZZ '}",2,1,71505,True,Completed,{'En': 'Continent'},66811,None,[],"{'type': 'Point', 'coordinates': [38.4313975, 1.4419683]}","{'En': 'place_kenya', 'He': 'מקום_קניה'}","{'En': 'Kenya', 'He': 'קניה'}",https://storage.googleapis.com/bhs-thumbnails/F71E3BB9-E61A-4994-B10A-4AF94CBE7C41.jpg,False
...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,None,[{'PictureUrl': 'https://storage.googleapis.com/bhs-flat-pics/06C18085-2C4B-4D85-8D0D-0995F2DFDC85.j ...,23731|41901|6268|46304|19203|492|14396|1704|19261|1200|19252|,"['image_members-of-the-faiz-family-take-refuge-in-sadah-yemen-1993', 'image_jewish-gold-and-silversm ...",2016-12-06T16:25:00,668000050062,haim,02360301.JPG|02673101.JPG|00578101.JPG|03436900.JPG|01882700.JPG|03787200.JPG|01392705.JPG|00120400. ...,Photos\00000793.scn\02360301.JPG|Photos\00000994.scn\02673101.JPG|Photos\00000440.scn\00578101.J

#,NegativeNumbers (string),IsLandscape (string),PrevPictureUnitsId (string),ExhibitionIsPreview (string),LocationInMuseum (any),Pictures (array),ToScan (string),PeriodDateTypeDesc (object),related (array),ExhibitionId (string),UpdateDate (string),OldPictureNumbers (string),OldUnitId (string),UpdateUser (string),PictureLocations (string),UnitDisplayStatus (integer),main_image_url (string),UnitPeriod (array),PrevPicturePaths (string),TS (string),PrevPictureId (string),PictureSources (string),UnitPersonalities (array),UnitTypeDesc (string),PeriodStartDate (string),PIctureReceivedIds (any),PeriodDateTypeCode (string),RightsDesc (string),OrderBy (any),Bibiliography (object),UnitText1 (object),UnitText2 (object),UnitPlaces (array),PictureFileNames (string),UnitStatus (integer),PersonalityIds (any),PeriodNum (string),PeriodDesc (object),UnitType (integer),UnitHeaderDMSoundex (object),PrevPictureFileNames (string),RightsCode (integer),PeriodTypeCode (string),UnitId (integer),PicId (string),IsValueUnit (integer),StatusDesc (string),PreviewPics (array),PicturePaths (string),Attachments (array),UserLexicon (string),EditorRemarks (string),Exhibitions (array),ForDisplay (string),DisplayStatusDesc (string),PeriodEndDate (string),PictureTypeCodes (string),PictureTypeDesc (object),Slug (object),Header (object),PeriodTypeDesc (object),thumbnail_url (string),ForPreview (integer),Resolutions (string),LocationCode (string)
1,|,0|,4765,0|,None,"[{'IsLandscape': '0', 'ForDisplay': '1', 'ToScan': '1', 'IsPreview': '1', 'PictureTypeDesc': {'En': ...",1|,"{'En': 'Year|', 'He': 'שנים|'}",['place_ethiopia'],112021|,2016-07-28T13:12:00,00479300|,4272,archive2,SLD-EXH.28.SS.75|,2,https://storage.googleapis.com/bhs-flat-pics/8C903B8B-F68B-457C-A9C4-9FEB876C373E.jpg,"[{'PeriodDateTypeDesc': {'En': 'Year', 'He': 'שנים'}, 'PeriodEndDate': '19649999', 'PeriodNum': '1', ...",Photos\00000711.scn\00427200.JPG|,000000000065f0e1,8C903B8B-F68B-457C-A9C4-9FEB876C373E,50027|,[],Photo,19640000|,None,1|,Full,None,"{'En': None, 'He': None}","{'En': 'The Shochet, Ethiopia, 1964\nRitual slaughtering is a duty carried out by the priest\nPhoto: ...","{'En': None, 'He': None}",[{'PlaceIds': '73463'}],00427200.JPG|,3,None,1|,"{'En': '1964|', 'He': '1964|'}",1,"{'En': 'ZZ E300000 ZZ E453000 E443000 ZZ E037000 ZZ ', 'He': 'ZZ H545300 H547530 ZZ H031700 H031750 ...",00427200.JPG,1,3|,4765,130059|,True,Completed,[{'PrevPictureId': '8C903B8B-F68B-457C-A9C4-9FEB876C373'}],Photos\00000711.scn\00427200.JPG,[],49343|56417|57248|,Picture - Onyx,"[{'ExhibitionId': '112021', 'ExhibitionIsPreview': '0'}]",1|,Museum only,19649999|,5|,"{'En': 'Slide-color|', 'He': 'שקופית - צבע|'}","{'En': 'image_the-shochet-ethiopia-1964', 'He': 'תמונה_השוחט-אתיופיה-1964'}","{'En': 'The Shochet, Ethiopia, 1964', 'He': 'השוחט, אתיופיה, 1964'}","{'En': 'Photo Date|', 'He': 'תאריך צילום|'}",https://storage.googleapis.com/bhs-thumbnails/8C903B8B-F68B-457C-A9C4-9FEB876C373E.jpg,False,100|,SLD-4183|
2,|,0|,4758,0|,None,"[{'IsLandscape': '0', 'ForDisplay': '1', 'ToScan': '1', 'IsPreview': '1', 'PictureTypeDesc': {'En': ...",1|,"{'En': 'Year|', 'He': 'שנים|'}",['place_ethiopia'],112021|,2016-07-28T12:13:00,00478601|,4265,archive2,EXH.28.SS.57|,2,https://storage.googleapis.com/bhs-flat-pics/62DBC71A-2237-434F-94DF-48C9317269E8.jpg,"[{'PeriodDateTypeDesc': {'En': 'Year', 'He': 'שנים'}, 'PeriodEndDate': '19649999', 'PeriodNum': '1', ...",Photos\00001105.scn\00426500.JPG|,000000000065f067,62DBC71A-2237-434F-94DF-48C9317269E8,50027|,[],Photo,19640000|,None,1|,Full,None,"{'En': None, 'He': None}","{'En': 'Hebrew class, Ethiopia, 1964\nPhoto: Eleonor Kahn\n(Beth Hatefutsoth Photo Archive)', 'He': ...","{'En': None, 'He': None}",[{'PlaceIds': '73463'}],00426500.JPG|,3,None,1|,"{'En': '1964|', 'He': '1964|'}",1,"{'En': 'ZZ E579700 ZZ E584000 E484000 ZZ E037000 ZZ ', 'He': 'ZZ H417900 H419000 ZZ H779300 H779400 ...",00426500.JPG,1,3|,4758,130047|,True,Completed,[{'PrevPictureId': '62DBC71A-2237-434F-94DF-48C9317269E'}],Photo

#,PeriodDateTypeDesc (object),FirstName (object),Title (object),LocationInMuseum (string),Pictures (array),PictureUnitsIds (string),related (array),Expr2 (string),Expr1 (string),UpdateDate (string),OldUnitId (string),IsMainCreatorType (string),PersonTypeCodesDesc (object),UpdateUser (string),UnitDisplayStatus (integer),UnitText2 (object),PrevPicturePaths (string),TS (string),FamilyNameIds (string),UnitPlaces (array),PictureSources (string),UnitTypeDesc (string),PeriodStartDate (string),Slug (object),PeriodDateTypeCode (string),RightsDesc (string),Bibiliography (object),UnitText1 (object),PersonTypeIds (string),MiddleName (object),main_image_url (string),UnitStatus (integer),LastName (object),PeriodDesc (object),PersonTypeCodes (string),UnitHeaderDMSoundex (object),PrevPictureFileNames (string),RightsCode (integer),PeriodTypeCode (string),UnitId (integer),IsValueUnit (integer),StatusDesc (string),NickName (object),UserLexicon (string),EditorRemarks (string),Attachments (array),DisplayStatusDesc (string),PeriodEndDate (string),PeriodNum (string),UnitType (integer),OtherNames (object),Header (object),PeriodTypeDesc (object),thumbnail_url (string),PersonalityId (integer),ForPreview (integer)
1,"{'En': 'Year|Year|', 'He': 'שנים|שנים|'}",{'He': 'Jacob'},{'He': ''},None,[{'PictureUrl': 'https://storage.googleapis.com/bhs-flat-pics/ADDD0651-71DD-4A06-8468-45E1D0EC7B2F.j ...,10999|11062|,"['image_jacques-faitlovitch-1881-1955-faitlovitch', 'familyname_faitlovitch', 'place_lodz']",פייטלוביץ',יעקב,2009-11-25T09:36:00,301,1|,"{'En': 'בלשן|', 'He': 'Linguist|'}",Zippi,2,"{'En': '', 'He': ''}",",Photos\00000152.scn\01051800.JPG|,Photos\00000153.scn\01058100.JPG|",     0&,80650|,[{'PlaceIds': '71959'}],50027|,Personality,18810000|19550000|,"{'En': 'luminary_jacques-faitlovich', 'He': 'אישיות_יעקב-זאק-פייטלוביץ'}",1|1|,Full,"{'En': None, 'He': None}","{'En': 'Dr. Jacques Faitlovitch (1881-1955) - an Orientalist, often called “Father of Falashas” (Je ...",1|,{'He': ''},https://storage.googleapis.com/bhs-flat-pics/99E7294B-0DB8-451A-B4A3-60F4E19CAB35.jpg,3,{'He': 'Faitlovich'},"{'En': '1881|1955|', 'He': '1881|1955|'}",189|,"{'En': 'ZZ E154000 E454000 E145400 E445400 ZZ E738750 E738740 ZZ ', 'He': 'ZZ H157000 ZZ H450000 ZZ ...",",01051800.JPG|,01058100.JPG|",1,1|2|,49752,True,Completed,{'He': 'Jacques'},93758,Miriam Sigalov - 25/11/2009,[],Museum only,18819999|19559999|,0|1|,8,{'He': ''},"{'En': 'Jacques Faitlovich', 'He': ""יעקב ז'אק פייטלוביץ'""}","{'En': 'Date of birth|Date of death|', 'He': 'לידה|פטירה|'}",https://storage.googleapis.com/bhs-thumbnails/99E7294B-0DB8-451A-B4A3-60F4E19CAB35.jpg,49752,False
2,"{'En': 'Year|Year|', 'He': 'שנים|שנים|'}",{'He': None},{'He': None},None,[],None,"['place_krzepice', 'place_los-angeles', 'place_paris', 'place_vienna']",None,None,2014-11-16T07:41:00,None,1|,"{'En': 'בלשן|', 'He': 'Linguist|'}",haim,2,"{'En': '', 'He': ''}",None,     .+,None,"[{'PlaceIds': '70523'}, {'PlaceIds': '71480'}, {'PlaceIds': '71957'}, {'PlaceIds': '72312'}, {'Place ...",None,Personality,19060000|20060000|,"{'En': 'luminary_leslau-wolf', 'He': 'אישיות_לסלאו-וולף'}",1|1|,Full,"{'En': 'Peter Reich\nHaim Ghiuzeli\n', 'He': None}","{'En': 'Leslau, Wolf (1906-2006), scholar of Semitic languages and leading world authority on the Se ...",1|,{'He': None},None,3,{'He': None},"{'En': '1906|2006|', 'He': '1906|2006|'}",189|,"{'En': 'ZZ E848000 ZZ E787000 ZZ ', 'He': 'ZZ H848000 H848700 ZZ H787000 ZZ '}",None,1,1|2|,157834,True,Completed,{'He': None},None,None,[],Museum only,19069999|20069999|,0|1|,8,{'He': None},"{'En': 'Leslau, Wolf', 'He': 'לסלאו, וולף'}","{'En': 'Date of birth|Date of death|', 'He': 'לידה|פטירה|'}",None,157834,False
...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,"{'En': 'Year|Year|', 'He': 'שנים|שנים|'}",{'He': ''},{'He': None},None,[],None,None,None,None,2009-01-28T13:31:00,None,1|0|,"{'En': 'מדינאים|חיילים|', 'He': 'Statesmen|Soldiers|'}",haim,3,"{'En': '', 'He': ''}",None,00000000000226b5,None,"[{'P

checkpoint saved: mojp-dbs-items


{'count_of_rows': 1600,
 'bytes': 5651145,
 'hash': 'ca52107d5287595b210020bdb10a510f',
 'dataset_name': None}

In [45]:
from dataflows import Flow, checkpoint, printer
from jewish_spotlight_dataflows.cached_apis import dump_cached_api


SKIP_EN_SLUGS = [
    'luminary_weizman-ezer'
]


def get_ethiopia_cached_api(package):
    package.pkg.add_resource({'name': 'ethiopia-cached-api', 'path': 'ethiopia-cached-api.csv', 'schema': {'fields': [
        {'name': field, 'type': 'string'} for field in [
            'UnitType', 'UnitTypeDesc', 'header_en', 'header_he', 'text_en', 'text_he',
            'slug_en', 'slug_he', 'video_url', 'main_image_url', 'preview_image_url',
            'image_urls', 'item_url_he', 'item_url_en',
        ]
    ]}})
    cached_api_rows = {}
    
    def _get_picture_url(picture_id):
        return f'https://storage.googleapis.com/bhs-flat-pics/{picture_id}.jpg'
    
    def _get_url_from_slug(lang, slug):
        if slug:
            collection, slug = slug.split('_')
            if lang == 'he':
                return f'https://dbs.bh.org.il/he/{slug}/{collection}'
            else:
                return f'https://dbs.bh.org.il/{collection}/{slug}'
        else:
            return None
    
    def _process_rows(rows):
        for row in rows:
            if rows.res.name in ['familyNames', 'photoUnits', 'personalities']:
                preview_image_url = None
                image_urls = []
                for picture in row['Pictures']:
                    picture_id = picture.get('PictureId')
                    if picture_id:
                        picture_url = _get_picture_url(picture['PictureId'])
                        image_urls.append(picture_url)
                        if picture.get('IsPreview') == '1':
                            preview_image_url = picture_url
                slug_en, slug_he = row['Slug'].get('En'), row['Slug'].get('He')
                if slug_en not in SKIP_EN_SLUGS:
                    cached_api_rows[row['UnitId']] = {
                        'UnitType': row['UnitType'],
                        'UnitTypeDesc': row['UnitTypeDesc'],
                        'header_en': row['Header']['En'],
                        'header_he': row['Header']['He'],
                        'text_en': row['UnitText1']['En'],
                        'text_he': row['UnitText1']['He'],
                        'slug_en': slug_en,
                        'slug_he': slug_he,
                        'video_url': row.get('video_url'),
                        'main_image_url': row.get('main_image_url'),
                        'preview_image_url': preview_image_url,
                        'image_urls': image_urls,
                        'item_url_he': _get_url_from_slug('he', slug_he),
                        'item_url_en': _get_url_from_slug('en', slug_en)
                    }
            yield row
    
    yield package.pkg
    for rows in package:
        yield _process_rows(rows)
    yield (r for r in cached_api_rows.values())

    
Flow(
    checkpoint('mojp-dbs-items'),
    get_ethiopia_cached_api,
    printer(tablefmt='html', resources=['ethiopia-cached-api']),
    dump_cached_api('ethiopia-cached-api', 'bhjs-content/places/ethiopia/cached-api.php'),
).process()[1]

using checkpoint data from .checkpoints/mojp-dbs-items


#,UnitType (string),UnitTypeDesc (string),header_en (string),header_he (string),text_en (string),text_he (string),slug_en (string),slug_he (string),video_url (string),main_image_url (string),preview_image_url (string),image_urls (string),item_url_he (string),item_url_en (string)
1,6,Family Name,HABESHA,חבשה,HABESHA This name is of Ethiopian origin. Habesha is a female personal name. It is the ancient n ...,HABESHA שם זה הוא ממקור אתיופי. חבשה הוא שם פרטי נשי. זהו שמה העתיק של אתיופיה (שבעבר נקראה חבש) ...,familyname_habesha,שםמשפחה_חבשה,None,None,None,[],https://dbs.bh.org.il/he/חבשה/שםמשפחה,https://dbs.bh.org.il/familyname/habesha
2,6,Family Name,AREGITU,ארגיטו,"AREGITU, AREGO, AREGNESH This name is of Ethiopian origin. Aregitu is a male personal name. It i ...","AREGITU, AREGO, AREGNESH שם זה הוא ממקור אתיופי. ארגיטו הוא שם פרטי גברי. זהו ביטוי אמהרי שפירוש ...",familyname_aregitu,שםמשפחה_ארגיטו,None,None,None,[],https://dbs.bh.org.il/he/ארגיטו/שםמשפחה,https://dbs.bh.org.il/familyname/aregitu
3,6,Family Name,TEWODROS,טוודרוס,TEWODROS This name is of Ethiopian origin. Tewodros is a male personal name. It is an Amharic ex ...,"TEWODROS שם זה הוא ממקור אתיופי. טוודרוס הוא שם פרטי גברי. זהו ביטוי אמהרי שפירושו ""להעריץ"". מי ...",familyname_tewodros,שםמשפחה_טוודרוס,None,None,None,[],https://dbs.bh.org.il/he/טוודרוס/שםמשפחה,https://dbs.bh.org.il/familyname/tewodros
4,6,Family Name,AYMELAK,איימלק,AYMELAK Surnames derive from one of many different origins. Sometimes there may be more than one ...,AYMELAK שמות משפחה נובעים מכמה מקורות שונים. לעיתים לאותו שם קיים יותר מהסבר אחד. שם משפחה זה הוא ...,familyname_aymelak,שםמשפחה_איימלק,None,None,None,[],https://dbs.bh.org.il/he/איימלק/שםמשפחה,https://dbs.bh.org.il/familyname/aymelak
5,6,Family Name,SOSNA (Ethiopian),סוסנה (שם אתיופי),SOSNA This name is of Ethiopian origin. Sosna is a female personal name. It is an Amharic express ...,"SOSNA שם זה הוא ממקור אתיופי. סוסנה הוא שם פרטי נשי. זהו ביטוי אמהרי שפירושו ""ורד"", ומקביל לשם הפ ...",familyname_sosna-ethiopian,שםמשפחה_סוסנה-שם-אתיופי,None,None,None,[],https://dbs.bh.org.il/he/סוסנה-שם-אתיופי/שםמשפחה,https://dbs.bh.org.il/familyname/sosna-ethiopian
6,6,Family Name,MELLE,מלה,"MELLE, MELES, MELESE Surnames derive from one of many different origins. Sometimes there may be m ...","MELLE, MELES, MELESE שמות משפחה נובעים מכמה מקורות שונים. לעיתים לאותו שם קיים יותר מהסבר אחד. שם ...",familyname_melle,שםמשפחה_מלה-84871,None,None,None,[],https://dbs.bh.org.il/he/מלה-84871/שםמשפחה,https://dbs.bh.org.il/familyname/melle
7,6,Family Name,CHAMOUILLI,שמולי,"CHAMOUILLI, CHEMOUILLI, CHEMOUILI Surnames derive from one of many different origins. Sometimes t ...","CHAMOUILLI, CHEMOUILLI, CHEMOUILI שמות משפחה נובעים מכמה מקורות שונים. לעיתים לאותו שם קיים יותר ...",familyname_chamouilli,שםמשפחה_שמולי,None,None,None,[],https://dbs.bh.org.il/he/שמולי/שםמשפחה,https://dbs.bh.org.il/familyname/chamouilli
8,6,Family Name,SCHMOLL,שמול,"SCHMOLL, SCHMOL, SCHMOLE Surnames derive from one of many different origins. Sometimes there may ...","SCHMOLL, SCHMOL, SCHMOLE שמות משפחה נובעים מכמה מקורות שונים. לעיתים לאותו שם קיים יותר מהסבר אחד ...",familyname_schmoll,שםמשפחה_שמול,None,None,None,[],https://dbs.bh.org.il/he/שמול/שםמשפחה,https://dbs.bh.org.il/familyname/schmoll
9,6,Family Name,SMOULSKI,סמולסקי,SMOULSKI Surnames derive from one of many different origins. Sometimes there may be more than one ...,SMOULSKI שמות משפחה נובעים מכמה מקורות שונים. לעיתים לאותו שם קיים יותר מהסבר אחד. שם משפחה זה הו ...,familyname_smoulski,שםמשפחה_סמולסקי,None,None,None,[],https://dbs.bh.org.il/he/סמולסקי/שםמשפחה,https://dbs.bh.org.il/familyname/smoulski
10,6,Family Name,SCHMOUL,שמול,"SCHMOUL, SCHMOULOWITCH Surnames derive from one of many different origins. Sometimes there may be ...","SCHMOUL, SCHMOULOWITCH שמות משפחה נובעים מכמה מקורות שונים. לעיתים לאותו שם קיים יותר מהסבר אחד. ...",familyname_schmoul,שםמשפחה_שמול-84867,None,None,None,[],https://dbs.bh.org.il/he/שמול-84867/שםמשפחה

{}